In [ ]:
import torch
import torch.nn as nn 
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

device  = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
data = pd.read_csv('riceClassification.csv').drop('id', axis = 1)
original_data = data.copy()
print(data.shape)
data.head(3)

cpu
(18185, 11)


,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1


In [21]:
## Verification target balanced
data.Class.value_counts()

Class
1    9985
0    8200
Name: count, dtype: int64

In [22]:
## Normalization 
for column in data.columns:
    data[column] = data[column] / data[column].abs().max()
data.head(3)

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.66661,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.53037,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.54638,0.856278,0.412994,0.959862,0.374747,1.0


In [24]:
X = np.array(data.iloc[:, : -1])   # (rows, columns)
y = np.array(data.iloc[:, -1])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42)

print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)
print(X_test.shape, y_test.shape)

(12729, 10) (12729,)
(2728, 10) (2728,)
(2728, 10) (2728,)


In [28]:
class dataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.tensor(X, dtype = torch.float32).to(device)
        self.Y = torch.tensor(Y, dtype = torch.float32).to(device)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.Y[index]
    
train_data = dataset(X_train, y_train)
valid_data = dataset(X_valid, y_valid)
test_data = dataset(X_test, y_test)

train_dataloader = DataLoader(train_data, batch_size = 8, shuffle = True)
valid_dataloader = DataLoader(valid_data, batch_size = 8, shuffle = True)
test_dataloader = DataLoader(test_data, batch_size = 8, shuffle = False)

In [ ]:
for x, y in train_dataloader:   # batch_size = 8
    print(x.shape)
    print(x)
    print('======')
    print(y.shape)
    print(y)
    break

torch.Size([8, 10])
tensor([[0.8762, 0.8691, 0.8776, 0.9211, 0.8330, 0.9361, 0.8457, 0.7510, 0.8521,
         0.5618],
        [0.6499, 0.8737, 0.6507, 0.9744, 0.6184, 0.8061, 0.7544, 0.7010, 0.7253,
         0.7618],
        [0.4334, 0.6305, 0.5943, 0.9364, 0.4101, 0.6583, 0.6875, 0.5311, 0.8426,
         0.6019],
        [0.3463, 0.5223, 0.5890, 0.8909, 0.3289, 0.5885, 0.8074, 0.4672, 0.8702,
         0.5031],
        [0.6259, 0.8275, 0.6584, 0.9656, 0.5923, 0.7911, 0.7412, 0.6678, 0.7697,
         0.7130],
        [0.5131, 0.7642, 0.5923, 0.9692, 0.4938, 0.7163, 0.6476, 0.6175, 0.7379,
         0.7320],
        [0.7995, 0.8449, 0.8260, 0.9286, 0.7634, 0.8942, 0.6293, 0.7265, 0.8308,
         0.5803],
        [0.6341, 0.9187, 0.6028, 0.9881, 0.5986, 0.7963, 0.8893, 0.7121, 0.6857,
         0.8647]])
torch.Size([8])
tensor([0., 1., 1., 1., 1., 1., 0., 1.])


In [ ]:
HIDDEN_NEURONS = 10
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.input = nn.Linear(in_features = X.shape[1], out_features = HIDDEN_NEURONS * 2)
        self.linear = nn.Linear(in_features = HIDDEN_NEURONS * 2, out_features = HIDDEN_NEURONS)
        self.linear2 = nn.Linear(in_features = HIDDEN_NEURONS, out_features = 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.input(x)
        x = self.linear(x)
        x = self.linear2(x)
        x = self.sigmoid(x)
        return x
    
model = MyModel().to(device)  # Only configure the model to device
summary(model, (X.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 20]             220
            Linear-2                   [-1, 10]             210
            Linear-3                    [-1, 1]              11
           Sigmoid-4                    [-1, 1]               0
Total params: 441
Trainable params: 441
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [38]:
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr = 0.001)

total_loss_train = []
total_loss_valid = []
total_accuracy_train = []
total_accuracy_valid = []

epochs = 10
for epoch in range(epochs):
    accuracy_train = 0
    accuracy_valid = 0

    loss_train = 0
    loss_valid = 0

    for data in train_dataloader:
        inputs, labels = data
        prediction = model(inputs).squeeze(1)
        print(prediction.shape)
        print(labels.shape)
        batch_loss = criterion(prediction, labels)

        break

    break

torch.Size([8])
torch.Size([8])
